# Bus241a Spring 2022: Problem set 7 starter file

Due Friday, March 18th at midnight

In [1]:
# Load helpers
# Will try to just load what I need on this
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor

# scaling
from sklearn.preprocessing import StandardScaler

# Cross-validation helpers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline

np.random.seed(121)

In [2]:
# Function to generate linear data experiments
def genLinData(N,M,noise):
    # y = x_1 + x_2 .. x_usedM + eps
    usedM = 25
    sigNoise = np.sqrt(1./usedM)
    rescale = 0.001
    X = np.random.normal(size=(N,M),loc=0,scale=sigNoise)
    eps = np.random.normal(size=N,loc=0,scale=noise)
    # build linear model
    y = np.sum(X[:,0:usedM],axis=1)+eps
    XMScale = X.copy()
    # rescale half of all predictors (every other one)
    XMScale[:,0::2] = rescale*X[:,0::2]
    # return both regular predictors, and rescaled ones (for model confusion)
    return X,XMScale, y

# 1. What is this experiment doing?

Look a little at the genLinData function.  What is is trying to do?  What is the difference between X and XMscale feature variables?

XMScale is scaled so the variance of explained part is same order as noise variance (if noise = 1)
X is the regular predictors without scalling and SMScale is rescaled predictors.

# 2. Theoretical R-squared

Generate a large sample of data using

X,Xs, y = genLinData(10000,100,1.0)

Report the usual mean train and test scores using a randomized cross validation with n_splits = 250, and test_size = 0.25.  Use just the (X,y) pair, and ignore Xs in this part.

This part is just to get you a feel for what the theoretical best R-squared would be which is the same as the R-squared for an infitely large sample.


In [3]:
X,Xs, y = genLinData(10000,100,1.0)

In [4]:

lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=250, test_size=.25)
CVInfo = cross_validate(lr, X, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.5115995589653072
0.49674805546202644


# 3. Linear regression

Now shift to a more difficult small sample using,

X, Xs, y = genLinData(500,100,1.0)

Use n_splits = 250, and test_size = 0.25 in a typical randomized cross validation for a linear regression model.  Report the mean score in the training and test sample.

To make sure you get exactly the same train/test splits, set the random_state parameter in your call to ShuffleSplit to 10.

Do this for for (X,y), and then do it for (Xs,y).  How do your results compare?


In [5]:
X, Xs, y = genLinData(500,100,1.0)


In [6]:
# For X,y

lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=250, test_size=.25,random_state=10)
CVInfo = cross_validate(lr, X, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.5977483458187949
0.2284480786075488


In [7]:
# For Xs,y
lr = LinearRegression()
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=250, test_size=.25,random_state=10)
CVInfo = cross_validate(lr, Xs, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.5977483458187949
0.22844807860754532


Both models have over fitting problem and generate almost identical low test and train sscores.Scaling has an impact on the magnitude of coefficients but it does not have any impact on the model predictions. Hence, in terms of getting the predictions, the 2 linear regression models are equivalent.

# 4. Ridge regression

Repeat exactly what you did in the last part, except now use a ridge regression with alpha = 10.
How do your results compare in the two cases? What does this tell you about the impact of data scales?

In [8]:
# For Xs,y
rd =  Ridge(alpha=10)
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=250, test_size=.25,random_state=10)
CVInfo = cross_validate(rd, Xs, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.27047452986044596
0.13283651217667927


In [9]:
# For X,y
rd =  Ridge(alpha=10)
# Set up randomized cross validation
shuffle = ShuffleSplit(n_splits=250, test_size=.25,random_state=10)
CVInfo = cross_validate(rd, X, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.4899742553338249
0.2806657759781399


In this case, the sacaled date, Xs, has minor over-fitting problem compared to the non-scaled data, X.
If the values of the features are closer to each other there are chances for the algorithm to get trained well and faster instead of the data set where the data points or features values have high differences with each other will take more time to understand the data and the accuracy will be lower. Ridge regression impose the penalty on the size of the coefficients. When the scales are different for the variables, they will have different contributions to the penalized terms since the penalized term is the sum of squares of all the coefficients. Hence, having the data on the same scale is important for Ridge regression.

## 5. KNN

Repeat exactly what you did in the past two sections for KNN regression (n_neighbors = 25).  Again, compare results.


In [22]:
knn = KNeighborsRegressor(n_neighbors=25)
nmc=250
# Again, scikit-learn tools
shuffle = ShuffleSplit(n_splits=nmc, test_size=.25)
CVInfo = cross_validate(knn, X, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.1579462811470064
0.08122936528739348


In [23]:
from sklearn.neighbors import KNeighborsClassifier
X, Xs, y = genLinData(500,100,1.0)
shuffle = ShuffleSplit(n_splits=250, test_size=.25)
knn = KNeighborsRegressor(n_neighbors=25)


CVInfo = cross_validate(knn, Xs, y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.13656514698583624
0.05823248119353205


Similar to Ridge regression, the mean train and test score for (Xs, y) pair is less than that of (X, y) pair. This might be because since KNN uses feature values to calculate the distance. The features with larger scale will have more weight and dominate the distance calculation.the values of the feature dimensions that have smaller range  may become uninformative and the algorithm would essentially rely on the single dimension whose range of values are substantially larger.So, normalization is important for knn model.

# 6. Ridge with a single StandardScaler() on the full sample

Try running the standard scaler across the full sample for the feature set Xs.  fit() and then transform().

Run Ridge on this new rescaled data with the alpha_fit = 10. hyperparameter.

Perform the same monte-carlo cross validation for this model as before. Print the mean score in the training and testing data.  Are your results getting closer to linear regression?

Print out the vector of standard deviations for your rescaled feature set.


In [24]:
scaler = StandardScaler()
scaler.fit(Xs)
scaler_tf = scaler.transform(Xs)
ridge = Ridge(alpha=10)
shuffle = ShuffleSplit(n_splits=250, test_size=.25)
CVInfo = cross_validate(ridge,scaler_tf,y, cv=shuffle,return_train_score=True)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.6381210118727696
0.31744643155338137


Yes, the results are getting closer to linear regression.

# 7. Pipeline

Now set this up with a formal pipeline built from StandardScaler() and Ridge(alpha=10).  Build 
the pipeline model, and run it through the same monte-carlo cross validation you have been doing.  Again do
this with the (Xs,y) pair.

Did your results change from the last run?  Is this model doing anything differently?

In [14]:
#method 1
from sklearn.pipeline import make_pipeline
shuffle = ShuffleSplit(n_splits=250, test_size=.25)
fullModel = make_pipeline(StandardScaler(), Ridge(alpha=10))
CVInfo = cross_validate(fullModel,Xs,y, cv=shuffle,return_train_score=True,n_jobs=-1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))


0.6425118042431737
0.347622380884018


In [15]:
#method 2
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge(alpha=10))
        ])

In [16]:
shuffle = ShuffleSplit(n_splits=250, test_size=.25)
CVInfo = cross_validate(pipe,Xs,y, cv=shuffle,return_train_score=True,n_jobs=-1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))


0.6421843540753434
0.3473696952879355


The results are very similar with the last run that set up with ridge with a single StandardScaler() on the full sample. This model is not doing anything differently because make_pipeline preprocesses with standard scaling and then ridge regression and it is doing the same set up as the last run.

# 8. GridSearch

Set up a formal scikit learn GridSearch for the best hyperparameter, alpha in the Ridge regression.

Build a pipeline model with the standard scaler.  Set up a search across the following list of alpha values,
[0.1, 0.25, 0.5, 1., 2., 5., 10.,25.,50.,100.,150.,200.]

Run the search using the same shuffle_split parameters that you have used, and the (Xs,y) pair.

Print a nice table of the (rank_test_score, mean_test_score, param_ridge_alpha).

Print the best_params.

Get the best model and perform your usual cross-validation on the (Xs,y) pair.



In [25]:
fullModel = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])

param_grid={'ridge__alpha':[0.1, 0.25, 0.5, 1., 2., 5., 10.,25.,50.,100.,150.,200.]}

shuffle_split = ShuffleSplit(test_size=0.25, n_splits=250)
# set up search
grid_search=GridSearchCV(fullModel,param_grid,cv=shuffle_split,
                              return_train_score=True,n_jobs=-1)

grid_search.fit(Xs,y)
# move results into DataFrame
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_ridge__alpha']])

    rank_test_score  mean_test_score param_ridge__alpha
0                12         0.307054                0.1
1                11         0.307346               0.25
2                10         0.307830                0.5
3                 9         0.308787                1.0
4                 8         0.310659                2.0
5                 7         0.315960                5.0
6                 6         0.323839               10.0
7                 5         0.341779               25.0
8                 3         0.358989               50.0
9                 1         0.369341              100.0
10                2         0.365649              150.0
11                4         0.356366              200.0


In [26]:
# Print best params and model
print("best param:",grid_search.best_params_)
print("best model:",grid_search.best_estimator_)
print("best test score:",grid_search.best_score_)

# This is best model
best_model = grid_search.best_estimator_

# Rerun cross validation for best model 
CVInfo = cross_validate(best_model, Xs, y, cv=shuffle,return_train_score=True)

print("(Xs, y) pair")
print("Mean Train Score:", np.mean(CVInfo['train_score']))
print("Mean Test Score:", np.mean(CVInfo['test_score']))

best param: {'ridge__alpha': 100.0}
best model: Pipeline(steps=[('scaler', StandardScaler()), ('ridge', Ridge(alpha=100.0))])
best test score: 0.36934067070109017
(Xs, y) pair
Mean Train Score: 0.6007680021654694
Mean Test Score: 0.3694136575099341


## Part 9: Grid search with formal train/validation/test split

Repeat your previous grid search, but this time follow the Ames real estate example by first splitting your data into a train_val part, and a test part.  Perform the grid search on the train_val part.  Report the best test set score from the grid_search (this is the mean of the monte-carlo cross validations).  Then take the best model.  Fit it to the entire train_val sample, and then report the score for the train_val sample, and also the test sample created at the start.

Compare performance between the final test sample, and the grid_search test.  Do you have any possible explation for the results.

In [27]:
X_trainValid, X_test, y_trainValid, y_test = train_test_split(Xs,y,test_size=0.25)
# implement search (again use trainValid data)
grid_search.fit(X_trainValid,y_trainValid)
# Print best params and model
print("best param:",grid_search.best_params_)
print("best model:",grid_search.best_estimator_)
print("best test score:",grid_search.best_score_)

best_model = grid_search.best_estimator_

# Rerun cross validation for this model just to check
CVInfo = cross_validate(best_model, X_trainValid, y_trainValid, cv=shuffle,return_train_score=True,n_jobs=-1)

print("Mean Train Score:",np.mean(CVInfo['train_score']))
print("Mean Test Score:",np.mean(CVInfo['test_score']))

best param: {'ridge__alpha': 100.0}
best model: Pipeline(steps=[('scaler', StandardScaler()), ('ridge', Ridge(alpha=100.0))])
best test score: 0.32357813270993185
Mean Train Score: 0.6157125708044423
Mean Test Score: 0.31739021818840896


In [29]:
# Try it on the final test sample
# You can now retrain on the full testValid sample
print("Final test sample:")
best_model.fit(X_trainValid,y_trainValid)
print(best_model.score(X_test,y_test))


Final test sample:
0.39274631861202913


The mean test score for the best model on the final test sample is sightly higher than that of the grid_search test. I think it might be because when I did the train_val/test split, the sets might not have the same underlying distribution which could result in the test score from the final test sample to be slightly better.